In [30]:
import ast
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
from sklearn.model_selection import train_test_split


In [31]:
raw_user_data = pd.read_json('../results/user_dataset.json')
raw_food_data = pd.read_json('../results/output.json')
listUserPreferences = pd.DataFrame(
    [data for data in raw_user_data.listUserPreferences])
listFoodPreferences = pd.DataFrame(
    [data for data in raw_food_data.food])
temp = pd.DataFrame(
    [data for data in listFoodPreferences.servings])
listFoodServings = pd.DataFrame(
    [item for sublist in temp.serving for item in sublist])
listFoodPreferences = listFoodPreferences.drop(columns=['servings'])
raw_listFood = listFoodPreferences.merge(
    listFoodServings, how='left', left_index=True, right_index=True)
listFood = raw_listFood.fillna(0)


In [32]:
listUserPreferences


,gender,weight,id_user,activityLevel,goals,height,disease,favoriteFood,birthdate
0,Male,50,3222915,1.9,0,180,[],"[Chicken Cordon Bleu, Chocolate Mousse Cake, S...",20-10-1990
1,Male,46,9930952,1.725,0,151,[obesity],"[Samosa, Pizza Dough, Baguette, Chocolate Mous...",10-09-1976
2,Male,60,3717984,1.725,0,164,"[heart, hypertension]","[Bubur Ayam, Mashed Potato, Puff Pastry, Chick...",01-05-2019
3,Male,41,6726690,1.55,0,156,"[diabetes, heart, obesity]","[Fish Cake, Red Velvet, Black Forest, Guacamol...",02-06-1979
4,Female,66,6452326,1.2,-500,151,[],"[Puff Pastry, Mashed Potato, Bubur Ayam, Okono...",22-03-1988
...,...,...,...,...,...,...,...,...,...
260,Male,65,7559844,1.55,500,163,[],"[Chicken Cordon Bleu, Pad Thai, Churros, Black...",26-01-1975
261,Male,84,8260717,1.9,500,196,[heart],"[Bibimbap, Puff Pastry, Thai Tea, Churros, Cin...",14-05-1950
262,Female,47,4895731,1.55,-500,180,"[diabetes, heart]","[Chocolate Lava Cake, Meringue Cookies, Takoya...",21-07-2016
263,Male,63,7540747,1.55,500,183,"[diabetes, hypertension, obesity]","[Thai Tea, Chocolate Mousse Cake, Pad Thai, Me...",22-10-1952


In [33]:
listFood


,food_id,food_name,food_type,food_url,brand_name,calcium,calories,carbohydrate,cholesterol,fat,...,serving_description,serving_id,serving_url,sodium,sugar,trans_fat,vitamin_a,vitamin_c,added_sugars,vitamin_d
0,2655309,Kebab,Generic,https://www.fatsecret.com/calories-nutrition/g...,0,176,620,77.10,53,16.63,...,1 kebab,2590791,https://www.fatsecret.com/calories-nutrition/g...,1029,6.91,0,37,22.2,0,0
1,58704864,Gyoza,Brand,https://www.fatsecret.com/calories-nutrition/h...,Hungry Planet,176,620,77.10,53,16.63,...,1 serving (390 g),2590792,https://www.fatsecret.com/calories-nutrition/g...,1029,6.91,0,37,22.2,0,0
2,68848666,Gyoza,Brand,https://www.fatsecret.com/calories-nutrition/y...,Yoshinoya,45,159,19.77,13,4.27,...,100 g,2590793,https://www.fatsecret.com/calories-nutrition/g...,264,1.77,0,9,5.7,0,0
3,515347,Gyoza,Brand,https://www.fatsecret.com/calories-nutrition/t...,Tokyo Joe's,13,45,5.60,4,1.21,...,1 oz,2590794,https://www.fatsecret.com/calories-nutrition/g...,75,0.50,0,3,1.6,0,0
4,1684073,Chicken Katsu,Brand,https://www.fatsecret.com/calories-nutrition/l...,L&L Hawaiian Barbecue,30,120,17.00,0,2.50,...,4 gyoza,48898797,https://www.fatsecret.com/calories-nutrition/h...,280,1.00,0,0,0,1.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,447317,Guacamole,Brand,https://www.fatsecret.com/calories-nutrition/c...,Chevy's Fresh Mex,0,330,41.00,0,15.00,...,1 serving,56535626,https://www.fatsecret.com/calories-nutrition/r...,390,6.00,0.500,0,0,0,0
261,2125933,Guacamole,Brand,https://www.fatsecret.com/calories-nutrition/c...,Costa Vida,0,100,12.00,0,4.50,...,1 croissant,44521664,https://www.fatsecret.com/calories-nutrition/p...,210,3.00,0,0,0,0,0
262,1995633,Guacamole,Brand,https://www.fatsecret.com/calories-nutrition/o...,Ortega,0,264,28.90,0,13.80,...,1 serving,56209720,https://www.fatsecret.com/calories-nutrition/t...,246,3.90,0.100,0,0,0,0
263,99383,Guacamole,Brand,https://www.fatsecret.com/calories-nutrition/c...,Chili's,0,250,25.00,35,14.00,...,1 serving,4438697,https://www.fatsecret.com/calories-nutrition/s...,230,3.00,0,0,0,0,0


In [34]:
# Convert the list columns to strings
listFood['food_name'] = listFood['food_name'].astype(str)
# listUserPreferences['favoriteFood'] = listUserPreferences['favoriteFood'].astype(str).apply(eval).apply(lambda x: ', '.join(x))
listUserPreferences = listUserPreferences.explode('favoriteFood')
listFood['food_name']


0              Kebab
1              Gyoza
2              Gyoza
3              Gyoza
4      Chicken Katsu
           ...      
260        Guacamole
261        Guacamole
262        Guacamole
263        Guacamole
264        Guacamole
Name: food_name, Length: 265, dtype: object

In [35]:

listUserPreferences['favoriteFood']


0        Chicken Cordon Bleu
0      Chocolate Mousse Cake
0                     Samosa
0               Cream Cheese
0                Carrot Cake
               ...          
264          Salmon Teriyaki
264               Burger Bun
264                 Pad Thai
264              Onion Rings
264                 Takoyaki
Name: favoriteFood, Length: 1981, dtype: object

In [36]:
# Merge the DataFrames based on the common attribute, such as food name or ID
raw_merged_df = pd.merge(listFood, listUserPreferences,
                         left_on='food_name', right_on='favoriteFood', how='inner')

raw_merged_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 11193 entries, 0 to 11192
Data columns (total 40 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   food_id                  11193 non-null  object
 1   food_name                11193 non-null  object
 2   food_type                11193 non-null  object
 3   food_url                 11193 non-null  object
 4   brand_name               11193 non-null  object
 5   calcium                  11193 non-null  object
 6   calories                 11193 non-null  object
 7   carbohydrate             11193 non-null  object
 8   cholesterol              11193 non-null  object
 9   fat                      11193 non-null  object
 10  fiber                    11193 non-null  object
 11  iron                     11193 non-null  object
 12  measurement_description  11193 non-null  object
 13  metric_serving_amount    11193 non-null  object
 14  metric_serving_unit      11193 non-nul

In [37]:
raw_merged_df

,food_id,food_name,food_type,food_url,brand_name,calcium,calories,carbohydrate,cholesterol,fat,...,vitamin_d,gender,weight,id_user,activityLevel,goals,height,disease,favoriteFood,birthdate
0,2655309,Kebab,Generic,https://www.fatsecret.com/calories-nutrition/g...,0,176,620,77.10,53,16.63,...,0,Male,43,3847302,1.55,0,196,"[obesity, diabetes, heart, hypertension]",Kebab,09-01-2019
1,2655309,Kebab,Generic,https://www.fatsecret.com/calories-nutrition/g...,0,176,620,77.10,53,16.63,...,0,Male,54,2935940,1.55,500,189,"[heart, diabetes, obesity, hypertension]",Kebab,26-08-1967
2,2655309,Kebab,Generic,https://www.fatsecret.com/calories-nutrition/g...,0,176,620,77.10,53,16.63,...,0,Male,40,5259626,1.2,0,199,"[heart, hypertension, obesity, diabetes]",Kebab,20-01-1972
3,2655309,Kebab,Generic,https://www.fatsecret.com/calories-nutrition/g...,0,176,620,77.10,53,16.63,...,0,Male,82,2097001,1.2,500,181,"[hypertension, diabetes, heart, obesity]",Kebab,20-03-1978
4,2655309,Kebab,Generic,https://www.fatsecret.com/calories-nutrition/g...,0,176,620,77.10,53,16.63,...,0,Male,84,7545290,1.375,-500,153,[diabetes],Kebab,21-04-2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11188,69852,Guacamole,Brand,https://www.fatsecret.com/calories-nutrition/f...,Freebirds,0,340,39.00,60,16.00,...,0,Female,92,8630293,1.2,0,159,[hypertension],Guacamole,28-12-2020
11189,69852,Guacamole,Brand,https://www.fatsecret.com/calories-nutrition/f...,Freebirds,0,340,39.00,60,16.00,...,0,Male,70,3959643,1.375,500,163,"[diabetes, obesity, heart]",Guacamole,27-02-2005
11190,69852,Guacamole,Brand,https://www.fatsecret.com/calories-nutrition/f...,Freebirds,0,340,39.00,60,16.00,...,0,Female,70,9023340,1.2,-500,191,"[hypertension, diabetes, heart, obesity]",Guacamole,24-11-1965
11191,69852,Guacamole,Brand,https://www.fatsecret.com/calories-nutrition/f...,Freebirds,0,340,39.00,60,16.00,...,0,Female,98,1433326,1.55,500,164,[obesity],Guacamole,25-06-1991


In [38]:
merged_df = raw_merged_df

merged_df['food_id'] = merged_df['food_id'].astype(str)
merged_df['food_name'] = merged_df['food_name'].astype(str)
merged_df['food_type'] = merged_df['food_type'].astype(str)
merged_df['food_url'] = merged_df['food_url'].astype(str)
merged_df['brand_name'] = merged_df['brand_name'].astype(str)
merged_df['calcium'] = merged_df['calcium'].astype(float)
merged_df['calories'] = merged_df['calories'].astype(float)
merged_df['carbohydrate'] = merged_df['carbohydrate'].astype(float)
merged_df['cholesterol'] = merged_df['cholesterol'].astype(float)
merged_df['fat'] = merged_df['fat'].astype(float)
merged_df['fiber'] = merged_df['fiber'].astype(float)
merged_df['iron'] = merged_df['iron'].astype(float)
merged_df['measurement_description'] = merged_df['measurement_description'].astype(str)
merged_df['metric_serving_amount'] = merged_df['metric_serving_amount'].astype(float)
merged_df['metric_serving_unit'] = merged_df['metric_serving_unit'].astype(str)
merged_df['monounsaturated_fat'] = merged_df['monounsaturated_fat'].astype(float)
merged_df['number_of_units'] = merged_df['number_of_units'].astype(float)
merged_df['polyunsaturated_fat'] = merged_df['polyunsaturated_fat'].astype(float)
merged_df['potassium'] = merged_df['potassium'].astype(float)
merged_df['protein'] = merged_df['protein'].astype(float)
merged_df['saturated_fat'] = merged_df['saturated_fat'].astype(float)
merged_df['serving_description'] = merged_df['serving_description'].astype(str)
merged_df['serving_id'] = merged_df['serving_id'].astype(str)
merged_df['serving_url'] = merged_df['serving_url'].astype(str)
merged_df['sodium'] = merged_df['sodium'].astype(float)
merged_df['sugar'] = merged_df['sugar'].astype(float)
merged_df['trans_fat'] = merged_df['trans_fat'].astype(float)
merged_df['vitamin_a'] = merged_df['vitamin_a'].astype(float)
merged_df['vitamin_c'] = merged_df['vitamin_c'].astype(float)
merged_df['added_sugars'] = merged_df['added_sugars'].astype(float)
merged_df['vitamin_d'] = merged_df['vitamin_d'].astype(float)
merged_df['gender'] = merged_df['gender'].astype(str)
merged_df['weight'] = merged_df['weight'].astype(float)
merged_df['goals'] = merged_df['goals'].astype(str)
merged_df['disease'] = merged_df['disease']
merged_df['favoriteFood'] = merged_df['favoriteFood'].astype(str)
merged_df['birthdate'] = merged_df['birthdate'].astype(str)

merged_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 11193 entries, 0 to 11192
Data columns (total 40 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   food_id                  11193 non-null  object 
 1   food_name                11193 non-null  object 
 2   food_type                11193 non-null  object 
 3   food_url                 11193 non-null  object 
 4   brand_name               11193 non-null  object 
 5   calcium                  11193 non-null  float64
 6   calories                 11193 non-null  float64
 7   carbohydrate             11193 non-null  float64
 8   cholesterol              11193 non-null  float64
 9   fat                      11193 non-null  float64
 10  fiber                    11193 non-null  float64
 11  iron                     11193 non-null  float64
 12  measurement_description  11193 non-null  object 
 13  metric_serving_amount    11193 non-null  float64
 14  metric_serving_unit   

In [39]:
#split the data into train and test sets with sklearn
train, test = train_test_split(merged_df, test_size=0.2)